![](./mitiq-logo.png)

[Mitiq](https://quantum-journal.org/papers/q-2022-08-11-774/) is a Python toolkit for implementing error mitigation techniques on quantum computers.

Current quantum computers are noisy due to interactions with the environment, imperfect gate applications, state preparation and measurement errors, etc. Error mitigation seeks to reduce these effects at the software level by compiling quantum programs in clever ways.

# Clifford Data Regression

Clifford data regression (CDR) is a learning-based quantum error mitigation technique in which an error mitigation model is trained with quantum circuits that _resemble_ the circuit of interest, but which are easier to classically simulate [1](https://link.aps.org/doi/10.1103/PhysRevResearch.3.033098), [2](http://dx.doi.org/10.22331/q-2021-11-26-592).

![](./cdr_workflow2_steps.png)

The CDR workflow Figure above shows a schema of the implementation of CDR in Mitiq. Similarly to ZNE and PEC, also CDR in Mitiq is divided in two main stages: The first one of circuit generation and the second for inference of the mitigated value. However, in CDR, the generation of quantum circuits is different, as it involves the generation of training circuits. The division of CDR into training, learning and prediction stages is shown in the figure below.


![](./cdr_diagram2.png)

Near-Clifford approximations of the actual circuit are simulated, without noise, on a classical simulator (circuits can be efficiently simulated classically) and executed on the noisy quantum computer (or a noisy simulator). These results are used as training data to infer the zero-noise expectation value of the error miitigated original circuit, that is finally run on the quantum computer (or noisy simulator)

# When should I use CDR?

## Advantages

The main advantage of CDR is that it can be applied without knowing the specific details of the noise
model. Indeed, in CDR, the effects of noise are indirectly _learned_ through the execution of an appropriate
set of test circuits. In this way, the final error mitigation inference tends to self-tune with respect
to the used backend.

This self-tuning property is even stronger in the case of _variable-noise-CDR_, i.e., when using the _scale_factors_ option
in {func}`.execute_with_cdr`. In this case, the final error mitigated expectation value is obtained
as a linear combination of noise-scaled expectation values. This is similar to the [ZNE approach](zne-5-theory.md) but, in CDR, 
the coefficients of the linear combination are learned instead of being fixed by the extrapolation model.


## Disadvantages

The main disadvantage of CDR is that the learning process is performed on a suite of test circuits which
only _resemble_ the original circuit of interest. Indeed, test circuits are _near-Clifford approximations_
of the original one. Only when the approximation is justified, the application of CDR can produce meaningful
results.
Increasing the `fraction_non_clifford` option in {func}`.execute_with_cdr` can alleviate this problem
to some extent. Note that, the larger `fraction_non_clifford` is, the larger the classical computation overhead is.

Another relevant aspect to consider is that, to apply CDR in a scalable way, a valid near-Clifford simulator
is necessary. Note that the computation cost of a valid near-Clifford simulator should scale with the number of non-Clifford
gates, independently from the circuit depth. Only in this case, the learning phase of CDR can be applied efficiently.


# What is the theory behind CDR?

Clifford data regression (CDR) is a quantum error mitigation technique that has been introduced in Ref. {cite}`Czarnik_2021_Quantum` and extended to variable-noise CDR in Ref. {cite}`Lowe_2021_PRR`.
. The presented error mitigation (EM) strategy is designed for gate-based quantum computers. This method primarily consists of creating a training data set $\{(X_{\phi_i}^{\text{error}}, X_{\phi_i}^{\text{exact}})\}$, where $X_{\phi_i}^{\text{error}}$ and $X_{\phi_i}^{\text{exact}}$ are the expectation values of an observable $X$ for a state $|\phi_i\rangle$ under error and error-free conditions, respectively. 

This method includes the following steps:

## Step 1: Choose Near-Clifford Circuits for Training

Near-Clifford circuits are selected due to their capability to be efficiently simulated classically, and are denoted by $S_\psi=\{|\phi_i\rangle\}_i$.

## Step 2: Construct the Training Set 

The training set $\{(X_{\phi_i}^{\text{error}}, X_{\phi_i}^{\text{exact}})\}_i$ is constructed by calculating the expectation values of $X$ for each state $|\phi_i\rangle$ in $S_\psi$, on both a quantum computer (to obtain $X_{\phi_i}^{\text{error}}$) and a classical computer (to obtain $X_{\phi_i}^{\text{exact}}$).

## Step 3: Learn the Error Mitigation Model

A model $f(X^{\text{error}}, a)$ for $X^{exact}$ is defined and learned. Here, $a$ is the set of parameters to be determined. This is achieved by minimizing the distance between the training set, as expressed by the following optimization problem:

$$a_{opt} = \underset{a}{\text{argmin}} \sum_i \left| X_{\phi_i}^{\text{exact}} - f(X_{\phi_i}^{\text{error}},a) \right|^2.$$

In this expression, $a_{opt}$ are the parameters that minimize the cost function.

## Step 4: Apply the Error Mitigation Model

Finally, the learned model $f(X^{\text{error}}, a_{opt})$ is used to correct the expectation values of $X$ for new quantum states, expressed as $X_\psi^{\text{exact}} = f(X_\psi^{\text{error}}, a_{opt})$.

The effectiveness of this method has been proven on circuits with up to 64 qubits and for tasks such as estimating ground-state energies. However, its performance is dependent on the task, the system, the quality of the training data, and the choice of model.



# How do I use CDR?

Here we show how to use CDR by means of a simple example.

In [1]:
import numpy as np
import warnings
warnings.simplefilter("ignore", np.ComplexWarning)

import cirq
from mitiq import cdr, Observable, PauliString


## Problem setup

To use CDR, we call {func}`.cdr.execute_with_cdr` with four "ingredients":

1. A quantum circuit to prepare a state $\rho$.
1. A quantum computer or noisy simulator to return a {class}`.QuantumResult` from $\rho$.
1. An observable $O$ which specifies what we wish to compute via $\text{Tr} [ \rho O ]$.
1. A near-Clifford (classical) circuit simulator.


### 1. Define a quantum circuit

The quantum circuit can be specified as any quantum circuit supported by Mitiq but
**must be compiled into a gateset in which the only non-Clifford gates are
single-qubit rotations around the $Z$ axis: $R_Z(\theta)$**.  For example:

$$\{ \sqrt{X}, R_Z(\theta), \text{CNOT}\},$$
$$\{{R_X(\pi/2)}, R_Z(\theta), \text{CZ}\},$$
$$\{H, S, R_Z(\theta), \text{CNOT}\},$$
$$ \dots$$

In the next cell we define (as an example) a quantum circuit which contains some
Clifford gates and some non-Clifford $R_Z(\theta)$ rotations.


In [2]:
a, b = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H.on(a), # Clifford
    cirq.H.on(b), # Clifford
    cirq.rz(1.75).on(a),
    cirq.rz(2.31).on(b),
    cirq.CNOT.on(a, b),  # Clifford
    cirq.rz(-1.17).on(b),
    cirq.rz(3.23).on(a),
    cirq.rx(np.pi / 2).on(a),  # Clifford
    cirq.rx(np.pi / 2).on(b),  # Clifford
)

# CDR works better if the circuit is not too short. So we increase its depth.
circuit = 5 * circuit

### Define an executor


In [3]:
from mitiq.interface.mitiq_cirq import compute_density_matrix

compute_density_matrix(circuit).round(3)


array([[ 0.834+0.j   , -0.027+0.075j, -0.004+0.061j, -0.022-0.052j],
       [-0.027-0.075j,  0.057-0.j   ,  0.007-0.006j, -0.003+0.002j],
       [-0.004-0.061j,  0.007+0.006j,  0.053+0.j   , -0.005+0.004j],
       [-0.022+0.052j, -0.003-0.002j, -0.005-0.004j,  0.056+0.j   ]],
      dtype=complex64)

### Observable
As an example, assume that we wish to compute the expectation value $\text{Tr} [ \rho O ]$ of the following observable $O$:

In [4]:
# Observable to measure.
obs = Observable(PauliString("ZZ"), PauliString("X", coeff=-1.75))
print(obs)

Z(q(0))*Z(q(1)) + (-1.75+0j)*X(q(0))


### (Near-Clifford) Simulator
The CDR method creates a set of "training circuits" which are related to the input circuit and are efficiently simulable. These circuits are simulated on a classical (noiseless) simulator to collect data for regression. The simulator should also return a `QuantumResult`.

In [5]:
def simulate(circuit: cirq.Circuit) -> np.ndarray:
    return compute_density_matrix(circuit, noise_level=(0.0,))

simulate(circuit).round(3)

array([[ 0.982-0.j   , -0.016+0.081j, -0.004+0.049j, -0.049-0.075j],
       [-0.016-0.081j,  0.007-0.j   ,  0.004-0.j   , -0.005+0.005j],
       [-0.004-0.049j,  0.004+0.j   ,  0.002+0.j   , -0.004+0.003j],
       [-0.049+0.075j, -0.005-0.005j, -0.004-0.003j,  0.008+0.j   ]],
      dtype=complex64)

# Run CDR
We first compute the noiseless result then the noisy result to compare to the mitigated result from CDR.

In [6]:
ideal_measurement = obs.expectation(circuit, simulate).real
print("ideal_measurement = ",ideal_measurement)

unmitigated_measurement = obs.expectation(circuit, compute_density_matrix).real
print("unmitigated_measurement = ", unmitigated_measurement)

mitigated_measurement = cdr.execute_with_cdr(
    circuit,
    compute_density_matrix,
    observable=obs,
    simulator=simulate,
    seed=0,
).real
print("mitigated_measurement = ", mitigated_measurement)

error_unmitigated = abs(unmitigated_measurement-ideal_measurement)
error_mitigated = abs(mitigated_measurement-ideal_measurement)

print("Error (unmitigated):", error_unmitigated)
print("Error (mitigated with CDR):", error_mitigated)

print("Relative error (unmitigated):", (error_unmitigated/ideal_measurement))
print("Relative error (mitigated with CDR):", error_mitigated/ideal_measurement)

print(f"Error reduction with CDR: {(error_unmitigated-error_mitigated)/error_unmitigated :.1%}.")


ideal_measurement =  1.0153722763061523
unmitigated_measurement =  0.8030939102172852
mitigated_measurement =  1.0307368605094165
Error (unmitigated): 0.2122783660888672
Error (mitigated with CDR): 0.015364584203264142
Relative error (unmitigated): 0.20906456778701882
Relative error (mitigated with CDR): 0.01513197135848473
Error reduction with CDR: 92.8%.
